In [1]:
# import faiss
# from langchain_community.docstore.in_memory import InMemoryDocstore
# from langchain_community.vectorstores.faiss import FAISS
import sys
sys.path.append('/workspace/mapping_tool')  # Assuming this is the root directory
from rag.bi_encoder import SAPEmbeddings
embeddings = SAPEmbeddings()
# index = faiss.IndexFlatL2(len(embeddings.embed_query("hello worlds")))

# vector_store = FAISS(
#     embedding_function=embeddings,
#     index=index,
#     docstore=InMemoryDocstore(),
#     index_to_docstore_id={},
# )

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
2024-09-06 20:08:12,349 - rag.utils - INFO - Semantic Type: True


In [2]:
from rag.utils import load_docs_from_jsonl
from uuid import uuid4
docs = load_docs_from_jsonl('/workspace/mapping_tool/data/output/sapbert_emb_docs_json.jsonl')
print(docs[0])



Opening file...


Loading Documents: 1000it [00:00, 1146.48it/s]

Total Unique Documents: 1000

page_content='concept name:alimentary tract and metabolism, synonyms:, domain:drug, concept Class:atc 1st, vocabulary:atc' metadata={'label': 'alimentary tract and metabolism', 'domain': 'drug', 'concept_class': 'atc 1st', 'vocab': 'atc', 'parent_term': '', 'scode': 'A', 'sid': '21600001', 'synonyms': '', 'has_answers': '', 'is_standard': 'C', 'vector': [[0.055979568511247635, -0.29887762665748596, 0.44734054803848267, -0.2205943465232849, 0.22514361143112183, -0.16349554061889648, 0.29436439275741577, 0.3582160770893097, -0.10512715578079224, 0.3218802809715271, -0.2537801265716553, -0.2312815934419632, 0.4727074205875397, 0.08076423406600952, -1.3948839902877808, 0.1963689774274826, -0.10144656896591187, -0.06201427802443504, 0.17835527658462524, 0.305111825466156, 0.11926195025444031, 0.21886682510375977, -0.33678650856018066, -0.5477790832519531, -0.3273129463195801, -0.2729000151157379, -0.24795779585838318, 0.07131101191043854, -0.03796863555908203, 

In [3]:
from langchain_qdrant import FastEmbedSparse, RetrievalMode
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
import qdrant_client.http.models as rest

sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm42-all-minilm-l6-v2-attentions")
vector_store = QdrantVectorStore.from_documents(
    docs,
    embedding=embeddings,
    url='komal.qdrant.137.120.31.148.nip.io', 
    port=443,
    https = True,
    vector_name ='omop_dense_vector',
    sparse_vector_name = 'omop_sparse_vector',
    sparse_embedding=sparse_embeddings,
    collection_name="icare4cvd_custom_collection",
    
    retrieval_mode=RetrievalMode.HYBRID,
    vector_params  = { 
    "size": 768,
    "distance": Distance.COSINE,
    "hnsw_config": rest.HnswConfigDiff(
            m=16,
            ef_construct=64,
            payload_m = 16 
        ),
    
    "quantization_config":rest.ScalarQuantization(
            scalar=rest.ScalarQuantizationConfig(
                type=rest.ScalarType.INT8,
                quantile=0.99,
                always_ram=True,
            ),
        ),
    "on_disk":True
    },
    sparse_vector_params = {
        "modifier":rest.Modifier.IDF,
        "index":{
            "full_scan_threshold":20000,
            "on_disk":True
        } 
    },
    force_recreate=True    
    
)



Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Embedding Documents: 100%|██████████| 1/1 [00:00<00:00,  7.41doc/s]


entity=dummy_text, synonyms=None, parent_term=None


0it [00:00, ?it/s]2024-09-06 20:08:14,599 - rag.utils - INFO - embed documents



entity=alimentary tract and metabolism, synonyms=None, parent_term=drug
entity=caries prophylactic agents, synonyms=None, parent_term=drug
entity=sodium fluoride; oral, local oral (caries prophylactic agents), synonyms=['domiphen / sodium chloride / sodium fluoride toothpaste', 'monofluorophosphate / sodium fluoride oral gel', 'hydrofluoric acid / sodium fluoride oral gel', 'sodium fluoride oral gel', 'carbamide peroxide / sodium fluoride oral gel', 'calcium carbonate / sodium fluoride oral tablet', 'calcium fluoride / sodium fluoride oral suspension', 'sodium fluoride oral solution', 'sodium fluoride / xylitol mouthwash', 'ascorbic acid / folic acid / niacinamide / riboflavin / sodium fluoride / thiamine / vitamin a / vitamin b 12 / vitamin b6 / vitamin d / vitamin e oral tablet'], parent_term=drug
entity=sodium monofluorophosphate; oral, local oral (caries prophylactic agents), synonyms=None, parent_term=drug
entity=olaflur; oral, local oral, synonyms=None, parent_term=drug
entity=st

...

entity=iron in combination with folic acid, synonyms=None, parent_term=drug
entity=ferrous amino acid complex; systemic, synonyms=['copper sulfate / ferrous fumarate / folic acid / manganese sulfate / niacinamide / pantothenate / polysaccharide iron complex / pyridoxine / riboflavin / sodium ascorbate / thiamine / vitamin b12 / zinc sulfate oral capsule', 'folic acid / polysaccharide iron complex oral tablet', 'polysaccharide iron complex oral tablet', 'ascorbic acid / cholecalciferol / cuprous oxide / folic acid / magnesium oxide / niacin / polysaccharide iron complex / potassium iodide / pyridoxine / riboflavin / thiamine / vitamin a / vitamin b12 / vitamin e / zinc oxide oral capsule', 'ferrous fumarate / folic acid / polysaccharide iron complex oral capsule', 'polysaccharide iron complex oral solution', 'calcium ascorbate / ferrous asparto glycinate / polysaccharide iron complex / succinic acid oral capsule', 'folic acid / polysaccharide iron complex oral solution', 'calcium ascorb

entity=iron, vitamin b12 and folic acid; systemic, synonyms=['iron, vitamin b12 and folic acid', 'iron', 'cobalt / copper sulfate / ferrous cation / folic acid / manganese / vitamin b 12 oral tablet', 'alpha tocopherol / ascorbic acid / biotin / calcium / chromium / copper / ergocalciferol / folic acid / iodine / iron / magnesium / manganese / molybdenum / niacinamide / pantothenic acid / phosphorus / potassium / pyridoxine / ... oral tablet', 'arnica extract / ascorbic acid / bromelains / calendula officinalis extract / chelidonium majus extract / ferrous cation / filipendula ulmaria flower extract / folic acid / hamamelis virginiana whole extract / magnesium / niacinamide / ... oral solution', 'ascorbic acid / biotin / calcium carbonate / choline / chromium, chelated / copper gluconate / ergocalciferol / folic acid / inositol / iodine / iron / magnesium oxide / methionine / niacinamide / pantothenate / potassium gluconate / ... oral tablet', 'ascorbic acid / calcium / cobalt / cupric

Embedding Documents: 100%|██████████| 40/40 [00:00<00:00, 40.54doc/s]


entity=cyanocobalamin tannin complex; parenteral, synonyms=None, parent_term=drug
entity=hydroxocobalamin; parenteral, synonyms=None, parent_term=drug
entity=cobamamide; systemic, synonyms=None, parent_term=drug
entity=mecobalamin; systemic, synonyms=None, parent_term=drug
entity=cyanocobalamin, combinations; systemic, synonyms=None, parent_term=drug
entity=hydroxocobalamin, combinations; systemic, synonyms=None, parent_term=drug
entity=folic acid and derivatives, synonyms=None, parent_term=drug
entity=folic acid; systemic (folic acid and derivatives), synonyms=None, parent_term=drug
entity=folic acid, combinations; systemic, synonyms=None, parent_term=drug
entity=other antianemic preparations, synonyms=None, parent_term=drug


16it [00:30,  1.93s/it]


In [4]:
results = vector_store.similarity_search(
    "alimentary canal",
    score_threshold=0.7,
    k=2
)
print(f"results: {results}")
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

2024-09-06 20:08:45,609 - rag.utils - INFO - Final Embedding: torch.Size([768])


results: [Document(metadata={'concept_class': 'atc 1st', 'domain': 'drug', 'has_answers': '', 'is_standard': 'C', 'label': 'alimentary tract and metabolism', 'parent_term': '', 'scode': 'A', 'sid': '21600001', 'synonyms': '', 'vector': [[0.055979568511247635, -0.29887762665748596, 0.44734054803848267, -0.2205943465232849, 0.22514361143112183, -0.16349554061889648, 0.29436439275741577, 0.3582160770893097, -0.10512715578079224, 0.3218802809715271, -0.2537801265716553, -0.2312815934419632, 0.4727074205875397, 0.08076423406600952, -1.3948839902877808, 0.1963689774274826, -0.10144656896591188, -0.06201427802443504, 0.17835527658462524, 0.305111825466156, 0.11926195025444032, 0.21886682510375977, -0.33678650856018066, -0.5477790832519531, -0.3273129463195801, -0.2729000151157379, -0.2479577958583832, 0.07131101191043854, -0.03796863555908203, -0.2083923816680908, 0.4726697206497192, 0.32677286863327026, 0.02321099489927292, 0.470431923866272, 0.3219035267829895, -0.3457314670085907, 0.256446

In [5]:
# uuids = [str(uuid4()) for _ in range(len(docs))]
# vector_store.add_documents(documents=docs, ids=uuids)